In [3]:
import numpy as np
import pandas as pd

import time
from datetime import datetime

In [4]:
merge_data_file = open('merge_data.csv', encoding = 'utf-8')
merge_data = pd.read_csv(merge_data_file)

data_row, data_col = merge_data.shape

merge_data['计划起飞时间戳'] = [time.mktime(datetime.strptime(item, '%Y-%m-%d %H:%M:%S').timetuple()) for item in merge_data['计划起飞时间']]
merge_data['计划飞行时间'] = (merge_data['计划到达时间'] - merge_data['计划起飞时间戳']) / 3600

In [5]:
def strsum(str_list):
    result = ''
    for item in str_list:
        if pd.isnull(item):
            pass
        else:
            result += item
    if result == '':
        result = np.nan
    return result

merge_data['机场_时间_航班编号'] = list(zip(merge_data['出发机场'], merge_data['计划起飞时间'], merge_data['航班编号']))
merge_data_group = merge_data.groupby('机场_时间_航班编号')

In [ ]:
merge_data_group_data_arrival = merge_data_group['计划到达时间'].agg([lambda x:x.iloc[0]]).rename(columns = {'<lambda>':'计划到达时间'})
merge_data_group_data_number = merge_data_group['飞机编号'].agg([lambda x:x.iloc[0]]).rename(columns = {'<lambda>':'飞机编号'})
merge_data_group_data_delay = merge_data_group['延误时长'].agg([lambda x:x.iloc[0]]).rename(columns = {'<lambda>':'延误时长'})
merge_data_group_data_plan_time = merge_data_group['计划飞行时间'].agg([lambda x:x.iloc[0]]).rename(columns = {'<lambda>':'计划飞行时间'})
merge_data_group_data_weather = merge_data_group['天气'].agg([lambda x:x.iloc[0]]).rename(columns = {'<lambda>':'天气'})
merge_data_group_data_low = merge_data_group['最低气温'].agg([lambda x:x.iloc[0]]).rename(columns = {'<lambda>':'最低气温'})
merge_data_group_data_high = merge_data_group['最高气温'].agg([lambda x:x.iloc[0]]).rename(columns = {'<lambda>':'最高气温'})
merge_data_group_data_y = merge_data_group['y'].agg([lambda x:x.iloc[0]]).rename(columns = {'<lambda>':'y'})

merge_data_group_data_emergency = merge_data_group['特情内容'].agg([lambda x:strsum(x)]).rename(columns = {'<lambda>':'特情内容'})
merge_data_group_data_emergency_nums = merge_data_group['特情内容'].agg([lambda x:sum(pd.notnull(x))]).rename(columns = {'<lambda>':'特情内容数量'})

merge_data_group_df = pd.DataFrame([])
merge_data_group_df['计划到达时间'] = merge_data_group_data_arrival.iloc[:,0]
merge_data_group_df['飞机编号'] = merge_data_group_data_number.iloc[:,0]
merge_data_group_df['延误时长'] = merge_data_group_data_delay.iloc[:,0]
merge_data_group_df['计划飞行时间'] = merge_data_group_data_plan_time.iloc[:,0]
merge_data_group_df['天气'] = merge_data_group_data_weather.iloc[:,0]
merge_data_group_df['最低气温'] = merge_data_group_data_low.iloc[:,0]
merge_data_group_df['最高气温'] = merge_data_group_data_high.iloc[:,0]
merge_data_group_df['特情内容'] = merge_data_group_data_emergency.iloc[:,0]
merge_data_group_df['特情内容数量'] = merge_data_group_data_emergency_nums.iloc[:,0]
merge_data_group_df['y'] = merge_data_group_data_y.iloc[:,0]

merge_data_group_df.to_csv('merge_data_group.csv', index = True)